In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = [[1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0],
        [0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0], 
        [0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 3, 0, 3]]

fig = plt.figure()
ax = fig.add_subplot(111)
ax.axes.get_yaxis().set_visible(False)
ax.set_aspect(1)

def avg(a, b):
    return (a + b) / 2.0

for y, row in enumerate(data):
    for x, col in enumerate(row):
        x1 = [x, x+1]
        y1 = np.array([y, y])
        y2 = y1+1
        if col == 1:
            plt.fill_between(x1, y1, y2=y2, color='red')
            plt.text(avg(x1[0], x1[1]), avg(y1[0], y2[0]), "A", 
                                        horizontalalignment='center',
                                        verticalalignment='center')
        if col == 2:
            plt.fill_between(x1, y1, y2=y2, color='orange')
            plt.text(avg(x1[0], x1[0]+1), avg(y1[0], y2[0]), "B", 
                                        horizontalalignment='center',
                                        verticalalignment='center')
        if col == 3:
            plt.fill_between(x1, y1, y2=y2, color='yellow')
            plt.text(avg(x1[0], x1[0]+1), avg(y1[0], y2[0]), "C", 
                                        horizontalalignment='center',
                                        verticalalignment='center')

plt.ylim(3, 0)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import string

x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
y = np.array([0, 0, 1, 0, 0, 0, 1, 1, 2, 2, 0, 2, 0, 2, 4])

plt.barh(y, [1]*len(x), left=x, color = 'red', edgecolor = 'red', align='center', height=1)

plt.ylim(max(y)+0.5, min(y)-0.5)

labels = np.array(list(string.uppercase))    
plt.yticks(np.arange(y.max()+1), labels)
plt.show()

In [ ]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import matplotlib.dates as md
import numpy as np
import datetime as dt
import time

n=20
duration=1000
now=time.mktime(time.localtime())
#print now
timestamps=np.linspace(now,now+duration,n)
print timestamps[9]
dates=[dt.datetime.fromtimestamp(ts) for ts in timestamps]
print dates[0]
values=np.sin((timestamps-now)/duration*2*np.pi)

plt.subplots_adjust(bottom=0.2)
plt.xticks( rotation=25 )
ax=plt.gca()
xfmt = md.DateFormatter('%Y-%m-%d %H:%M:%S')
ax.xaxis.set_major_formatter(xfmt)
plt.plot(dates,values)
plt.show()

In [ ]:
import json
import datetime as dt
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import numpy as np
import string

# Read file
# is there a more elegant way? What if linewise send per ajax-post?
f = open('profile.log', 'r')
data_json = f.readlines()
f.close()
data = [json.loads(line) for line in data_json]


# Convert all string times to dateTime obj
# Helper function for string conversion
def make_dt(data_line):
    return dt.datetime.strptime(data_line['time'], "%Y-%m-%d %H:%M:%S,%f") #2003-07-08 16:49:45,896
# Do converstion
for data_line in data:
    data_line['time'] = make_dt(data_line)

    
# Resort for plotting
profile = []
for event in data:
    if event['is_start']:
        profile.append({'name':[event['name']],
                        'start':event['time']})
    else:
        stopping_event = next(ev for ev in profile if ev['name'][0] == event['name'])
        stopping_event.update({'end':event['time'],
                               'duration': event['time'] - stopping_event['start']})
#print profile

In [ ]:
import svgwrite
from __future__ import division
# px
height = 800
width = 1500 # including 500 px for text at the end
offx = 10
offy = 10
barheight = 30
filename = "skill_profile.svg"


first_time = profile[0]['start']
max_duration = profile[-1]['end']-profile[0]['start']

# Make this constant if plots need to be visible comparable
divider = 1.0
print "Find longest duration:"
divider = conv_duration(max_duration) / (width-2*offx-500) # 500 px for text at the end

# convert duration to number
def conv_duration(duration):
    global max_duration, divider
    microsec = duration.total_seconds() * 1000.0 / divider
    # TODO make better # find global divider
    print "Duration: ", microsec
    return microsec

def conv_start(starttime):
    global first_time, divider
    microsec = (starttime - first_time).total_seconds() * 1000.0 / divider
    print "Start:", microsec
    return microsec


names = []
duration = []
start_time = []
for event in profile:
    names.append(event['name'][0])
    print event['name'][0]
    start_time.append(conv_start(event['start']))
    duration.append(conv_duration(event['duration']))
    print "End:", start_time[-1]+duration[-1]



svg_document = svgwrite.Drawing(filename = filename,
                                size = (width, height))

substractor = 0
for i in range(len(names)):
    # Hack to make multiple loops useful
    if names[i][0:len(names[i])-16] == "pitasc_root":
        substractor = i

    svg_document.add(
        svg_document.rect(
            insert = (offx + start_time[i], height-offy-(i+1-substractor)*barheight),
            size = (duration[i], barheight),
            stroke_width = "1",
            stroke = "black",
            fill = "rgb(0, 187, 145)"))
    svg_document.add(
        svg_document.text(
            names[i][0:len(names[i])-16]
                + " : " + str(duration[i]) + "ms",
            insert = (offx + start_time[i] + 2, 
                      height-offy-(i-substractor)*barheight - 7))) # pixel magic for text...

svg_document.save()